# Using custom containers with AI Platform Training

**Learning Objectives:**
1. Learn how to create a train and a validation split with Big Query
1. Learn how to wrap a machine learning model into a Docker container and train in on CAIP
1. Learn how to use the hyperparameter tunning engine on GCP to find the best hyperparameters
1. Learn how to deploy a trained machine learning model GCP as a rest API and query it

In this lab, you develop, package as a docker image, and run on **AI Platform Training** a training application that trains a multi-class classification model that predicts the type of forest cover from cartographic data. The [dataset](../../../datasets/covertype/README.md) used in the lab is based on **Covertype Data Set** from UCI Machine Learning Repository.

The training code uses `scikit-learn` for data pre-processing and modeling. The code has been instrumented using the `hypertune` package so it can be used with **AI Platform** hyperparameter tuning.


In [1]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## Configure environment settings

Set location paths, connections strings, and other environment settings. Make sure to update   `REGION`, and `ARTIFACT_STORE`  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.

In [19]:
!(gsutil ls | grep kubeflow)

gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/


In [33]:
REGION = 'us-central1'
ARTIFACT_STORE = "gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default"

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
os.environ['PROJECT_ID'] = PROJECT_ID

DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'dataset.csv')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'dataset.csv')

## Importing the dataset into BigQuery

In [9]:
%%bash

DATASET_LOCATION=US
DATASET_ID=covertype_dataset
TABLE_ID=covertype
DATA_SOURCE=gs://workshop-datasets/covertype/small/dataset.csv
SCHEMA=Elevation:INTEGER,\
Aspect:INTEGER,\
Slope:INTEGER,\
Horizontal_Distance_To_Hydrology:INTEGER,\
Vertical_Distance_To_Hydrology:INTEGER,\
Horizontal_Distance_To_Roadways:INTEGER,\
Hillshade_9am:INTEGER,\
Hillshade_Noon:INTEGER,\
Hillshade_3pm:INTEGER,\
Horizontal_Distance_To_Fire_Points:INTEGER,\
Wilderness_Area:STRING,\
Soil_Type:STRING,\
Cover_Type:INTEGER

bq --location=$DATASET_LOCATION --project_id=$PROJECT_ID mk --dataset $DATASET_ID

bq --project_id=$PROJECT_ID --dataset_id=$DATASET_ID load \
--source_format=CSV \
--skip_leading_rows=1 \
--replace \
$TABLE_ID \
$DATA_SOURCE \
$SCHEMA

Dataset 'qwiklabs-gcp-01-3aff5ef1f764:covertype_dataset' successfully created.


Waiting on bqjob_r6dfbf5b1884062c4_00000176091a051e_1 ... (1s) Current status: DONE   


## Explore the Covertype dataset 

In [10]:
%%bigquery
SELECT *
FROM `covertype_dataset.covertype`

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2085,256,18,150,27,738,176,248,208,914,Cache,C2702,5
1,2125,256,20,30,12,871,169,248,215,300,Cache,C2702,2
2,2146,256,34,150,62,1253,122,237,239,511,Cache,C2702,2
3,2186,256,38,210,102,1294,109,232,244,552,Cache,C2702,2
4,2831,256,25,277,183,1706,153,246,225,1485,Commanche,C2705,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3136,254,12,319,60,5734,193,248,193,2467,Rawah,C7746,1
99996,3242,254,12,636,148,3551,193,248,193,2010,Commanche,C7757,0
99997,2071,255,12,234,63,342,192,247,193,247,Cache,C2706,2
99998,3248,255,12,730,113,725,192,247,193,2724,Commanche,C7756,1


## Create training and validation splits

Use BigQuery to sample training and validation splits and save them to GCS storage
### Create a training split

In [11]:
!bq query \
-n 0 \
--destination_table covertype_dataset.training \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `covertype_dataset.covertype` AS cover \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), 10) IN (1, 2, 3, 4)' 

Waiting on bqjob_r1f118a07c7931deb_00000176091bf76d_1 ... (1s) Current status: DONE   


In [26]:
!bq extract \
--destination_format CSV \
covertype_dataset.training \
$TRAINING_FILE_PATH

Waiting on bqjob_r467340c00c31af4e_0000017609258d98_1 ... (0s) Current status: DONE   


### Create a validation split

In [27]:
!bq query \
-n 0 \
--destination_table covertype_dataset.validation \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `covertype_dataset.covertype` AS cover \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), 10) IN (8)' 

Waiting on bqjob_r168b960872e0262e_000001760925d6e3_1 ... (0s) Current status: DONE   


In [28]:
!bq extract \
--destination_format CSV \
covertype_dataset.validation \
$VALIDATION_FILE_PATH

Waiting on bqjob_r16979947fff573bf_000001760925ec98_1 ... (0s) Current status: DONE   


In [34]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)
print(df_train.shape)
print(df_validation.shape)

(40009, 13)
(9836, 13)


## Develop a training application

### Configure the `sklearn` training pipeline.

The training pipeline preprocesses data by standardizing all numeric features using `sklearn.preprocessing.StandardScaler` and encoding all categorical features using `sklearn.preprocessing.OneHotEncoder`. It uses stochastic gradient descent linear classifier (`SGDClassifier`) for modeling.

In [35]:
numeric_feature_indexes = slice(0, 10)
categorical_feature_indexes = slice(10, 12)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log', tol=1e-3))
])

### Convert all numeric features to `float64`

To avoid warning messages from `StandardScaler` all numeric features are converted to `float64`.

In [36]:
num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}

df_train = df_train.astype(num_features_type_map)
df_validation = df_validation.astype(num_features_type_map)

### Run the pipeline locally.

In [37]:
X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']
X_validation = df_validation.drop('Cover_Type', axis=1)
y_validation = df_validation['Cover_Type']

pipeline.set_params(classifier__alpha=0.001, classifier__max_iter=200)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  slice(0, 10, None)),
                                                 ('cat',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64

### Calculate the trained model's accuracy.

In [38]:
accuracy = pipeline.score(X_validation, y_validation)
print(accuracy)

0.695201301342009


### Prepare the hyperparameter tuning application.
Since the training run on this dataset is computationally expensive you can benefit from running a distributed hyperparameter tuning job on AI Platform Training.

In [39]:
TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

### Write the tuning script. 

Notice the use of the `hypertune` package to report the `accuracy` optimization metric to AI Platform hyperparameter tuning service.

In [40]:
%%writefile {TRAINING_APP_FOLDER}/train.py

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
    df_train = pd.read_csv(training_dataset_path)
    df_validation = pd.read_csv(validation_dataset_path)

    if not hptune:
        df_train = pd.concat([df_train, df_validation])

    numeric_feature_indexes = slice(0, 10)
    categorical_feature_indexes = slice(10, 12)

    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', SGDClassifier(loss='log',tol=1e-3))
    ])

    num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
    df_train = df_train.astype(num_features_type_map)
    df_validation = df_validation.astype(num_features_type_map) 

    print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
    X_train = df_train.drop('Cover_Type', axis=1)
    y_train = df_train['Cover_Type']

    pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
    pipeline.fit(X_train, y_train)

    if hptune:
        X_validation = df_validation.drop('Cover_Type', axis=1)
        y_validation = df_validation['Cover_Type']
        accuracy = pipeline.score(X_validation, y_validation)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
          hyperparameter_metric_tag='accuracy',
          metric_value=accuracy
        )

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(pipeline, model_file)
        gcs_model_path = "{}/{}".format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
        print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
    fire.Fire(train_evaluate)

Writing training_app/train.py


### Package the script into a docker image.

Notice that we are installing specific versions of `scikit-learn` and `pandas` in the training image. This is done to make sure that the training runtime is aligned with the serving runtime. Later in the notebook you will deploy the model to AI Platform Prediction, using the 1.15 version of AI Platform Prediction runtime. 

Make sure to update the URI for the base image so that it points to your project's **Container Registry**.

In [41]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing training_app/Dockerfile


### Build the docker image. 

You use **Cloud Build** to build the image and push it your project's **Container Registry**. As you use the remote cloud service to build the image, you don't need a local installation of Docker.

In [42]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [43]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 3.2 KiB before compression.
Uploading tarball of [training_app] to [gs://qwiklabs-gcp-01-3aff5ef1f764_cloudbuild/source/1606471362.488355-ef031f4520d94f5587c7ac968a1e409d.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-01-3aff5ef1f764/builds/6d2b1a79-d31d-4c53-977f-5dff7edc955b].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/6d2b1a79-d31d-4c53-977f-5dff7edc955b?project=1016448934670].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "6d2b1a79-d31d-4c53-977f-5dff7edc955b"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-01-3aff5ef1f764_cloudbuild/source/1606471362.488355-ef031f4520d94f5587c7ac968a1e409d.tgz#1606471363290397
Copying gs://qwiklabs-gcp-01-3aff5ef1f764_cloudbuild/source/1606471362.488355-ef031f4520d94f5587c7ac968a1e409d.tgz#1606471363290397...
/ [1 files][  1.5 KiB/  1.5 KiB]                                    

## Submit an AI Platform hyperparameter tuning job

### Create the hyperparameter configuration file. 
Recall that the training code uses `SGDClassifier`. The training application has been designed to accept two hyperparameters that control `SGDClassifier`:
- Max iterations
- Alpha

The below file configures AI Platform hypertuning to run up to 6 trials on up to three nodes and to choose from two discrete values of `max_iter` and the linear range betwee 0.00001 and 0.001 for `alpha`.

In [44]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 4
    maxParallelTrials: 4
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_iter
      type: DISCRETE
      discreteValues: [
          200,
          500
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.00001
      maxValue:  0.001
      scaleType: UNIT_LINEAR_SCALE

Writing training_app/hptuning_config.yaml


### Start the hyperparameter tuning job.

Use the `gcloud` command to start the hyperparameter tuning job.

In [45]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--hptune

Job [JOB_20201127_100720] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20201127_100720

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20201127_100720
jobId: JOB_20201127_100720
state: QUEUED


### Monitor the job.

You can monitor the job using GCP console or from within the notebook using `gcloud` commands.

In [46]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-11-27T10:07:22Z'
etag: 8ZoQzGHn_Ng=
jobId: JOB_20201127_100720
startTime: '2020-11-27T10:07:25Z'
state: RUNNING
trainingInput:
  args:
  - --training_dataset_path=gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/data/training/dataset.csv
  - --validation_dataset_path=gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/data/validation/dataset.csv
  - --hptune
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 4
    maxTrials: 4
    params:
    - discreteValues:
      - 200.0
      - 500.0
      parameterName: max_iter
      type: DISCRETE
    - maxValue: 0.001
      minValue: 1e-05
      parameterName: alpha
      scaleType: UNIT_LINEAR_SCALE
      type: DOUBLE
  jobDir: gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/jobs/JOB_20201127_100720
  masterConfig:
    imageUri: gcr.io/qwiklabs-gcp-01-3aff5ef1f764/trainer_image:latest
  region: us-central1
trainin

In [47]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

INFO	2020-11-27 10:07:22 +0000	service		Validating job requirements...
INFO	2020-11-27 10:07:22 +0000	service		Job creation request has been successfully validated.
INFO	2020-11-27 10:07:23 +0000	service		Job JOB_20201127_100720 is queued.
INFO	2020-11-27 10:08:02 +0000	service	2	Waiting for job to be provisioned.
INFO	2020-11-27 10:08:02 +0000	service	4	Waiting for job to be provisioned.
INFO	2020-11-27 10:08:03 +0000	service	1	Waiting for job to be provisioned.
INFO	2020-11-27 10:08:03 +0000	service	3	Waiting for job to be provisioned.
INFO	2020-11-27 10:08:06 +0000	service	4	Waiting for training program to start.
INFO	2020-11-27 10:08:06 +0000	service	2	Waiting for training program to start.
INFO	2020-11-27 10:08:06 +0000	service	3	Waiting for training program to start.
INFO	2020-11-27 10:08:06 +0000	service	1	Waiting for training program to start.
INFO	2020-11-27 10:11:16 +0000	master-replica-0	4	Starting training: alpha=0.0005050522056210883, max_iter=500
INFO	2020-11-27 10:11:16 

### Retrieve HP-tuning results.

After the job completes you can review the results using GCP Console or programatically by calling the AI Platform Training REST end-point.

In [48]:
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response

{'jobId': 'JOB_20201127_100720',
 'trainingInput': {'args': ['--training_dataset_path=gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/data/training/dataset.csv',
   '--validation_dataset_path=gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/data/validation/dataset.csv',
   '--hptune'],
  'hyperparameters': {'goal': 'MAXIMIZE',
   'params': [{'parameterName': 'max_iter',
     'type': 'DISCRETE',
     'discreteValues': [200, 500]},
    {'parameterName': 'alpha',
     'minValue': 1e-05,
     'maxValue': 0.001,
     'type': 'DOUBLE',
     'scaleType': 'UNIT_LINEAR_SCALE'}],
   'maxTrials': 4,
   'maxParallelTrials': 4,
   'hyperparameterMetricTag': 'accuracy',
   'enableTrialEarlyStopping': True},
  'region': 'us-central1',
  'jobDir': 'gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/jobs/JOB_20201127_100720',
  'masterConfig': {'imageUri': 'gcr.io/qwiklabs-gcp-01-3aff5ef1f764/trainer_image:latest'}},
 'createTime': '2020-11-27T10:07:22Z',
 'startTime': '

The returned run results are sorted by a value of the optimization metric. The best run is the first item on the returned list.

In [49]:
response['trainingOutput']['trials'][0]

{'trialId': '4',
 'hyperparameters': {'alpha': '0.00050505220562108827', 'max_iter': '500'},
 'finalMetric': {'trainingStep': '1', 'objectiveValue': 0.7017080113867425},
 'startTime': '2020-11-27T10:08:02.893697256Z',
 'endTime': '2020-11-27T10:14:22Z',
 'state': 'SUCCEEDED'}

## Retrain the model with the best hyperparameters

You can now retrain the model using the best hyperparameters and using combined training and validation splits as a training dataset.

### Configure and run the training job

In [50]:
alpha = response['trainingOutput']['trials'][0]['hyperparameters']['alpha']
max_iter = response['trainingOutput']['trials'][0]['hyperparameters']['max_iter']

In [51]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--alpha=$alpha \
--max_iter=$max_iter \
--nohptune

Job [JOB_20201127_102429] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20201127_102429

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20201127_102429
jobId: JOB_20201127_102429
state: QUEUED


In [54]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

INFO	2020-11-27 10:24:31 +0000	service		Validating job requirements...
INFO	2020-11-27 10:24:31 +0000	service		Job creation request has been successfully validated.
INFO	2020-11-27 10:24:32 +0000	service		Waiting for job to be provisioned.
INFO	2020-11-27 10:24:32 +0000	service		Job JOB_20201127_102429 is queued.
INFO	2020-11-27 10:24:34 +0000	service		Waiting for training program to start.
INFO	2020-11-27 10:27:51 +0000	master-replica-0		Copying file://model.pkl [Content-Type=application/octet-stream]...
INFO	2020-11-27 10:27:51 +0000	master-replica-0		/ [0 files][    0.0 B/  6.2 KiB]                                                
INFO	2020-11-27 10:27:51 +0000	master-replica-0		/ [1 files][  6.2 KiB/  6.2 KiB]                                                
INFO	2020-11-27 10:27:51 +0000	master-replica-0		Operation completed over 1 objects/6.2 KiB.                                      
INFO	2020-11-27 10:27:51 +0000	master-replica-0		Starting training: alpha=0.0005050522056210883, m

### Examine the training output

The training script saved the trained model as the 'model.pkl' in the `JOB_DIR` folder on GCS.

In [53]:
!gsutil ls $JOB_DIR

gs://qwiklabs-gcp-01-3aff5ef1f764-kubeflowpipelines-default/jobs/JOB_20201127_102429/model.pkl


## Deploy the model to AI Platform Prediction

### Create a model resource

In [55]:
model_name = 'forest_cover_classifier'
labels = "task=classifier,domain=forestry"

!gcloud ai-platform models create  $model_name \
 --region=$REGION \
 --labels=$labels

Using endpoint [https://ml.googleapis.com/]
Created ml engine model [projects/qwiklabs-gcp-01-3aff5ef1f764/models/forest_cover_classifier].


### Create a model version

In [58]:
model_version = 'v01'

!gcloud ai-platform versions create {model_version} \
 --model={model_name} \
 --origin=$JOB_DIR \
 --runtime-version=1.15 \
 --framework=scikit-learn \
 --python-version=3.7

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


### Serve predictions
#### Prepare the input file with JSON formated instances.

In [59]:
input_file = 'serving_instances.json'

with open(input_file, 'w') as f:
    for index, row in X_validation.head().iterrows():
        f.write(json.dumps(list(row.values)))
        f.write('\n')

In [60]:
!cat $input_file

[2841.0, 45.0, 0.0, 644.0, 282.0, 1376.0, 218.0, 237.0, 156.0, 1003.0, "Commanche", "C4758"]
[2494.0, 180.0, 0.0, 0.0, 0.0, 819.0, 219.0, 238.0, 157.0, 5531.0, "Rawah", "C6101"]
[3153.0, 90.0, 0.0, 335.0, 11.0, 5842.0, 219.0, 237.0, 155.0, 930.0, "Rawah", "C7101"]
[2907.0, 0.0, 0.0, 560.0, 35.0, 3027.0, 218.0, 238.0, 156.0, 2054.0, "Rawah", "C7745"]
[2916.0, 0.0, 0.0, 0.0, 0.0, 4562.0, 218.0, 238.0, 156.0, 5442.0, "Rawah", "C7745"]


#### Invoke the model

In [61]:
!gcloud ai-platform predict \
--model $model_name \
--version $model_version \
--json-instances $input_file

Using endpoint [https://ml.googleapis.com/]
[1, 1, 0, 1, 1]


<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>